# Roman Mining Sites: Data Science Analysis

**Dataset:** Oxford Roman Economy Project (OXREP) Mines Database (1,399 sites × 47 features)  
**Framework:** CRISP-DM (Cross-Industry Standard Process for Data Mining)

---

## Business Questions

This analysis answers the following research questions about Roman mining operations:

### Question 1: Which regions had the most Roman mining activity?
**Approach:** Descriptive statistics and visualization  
**Goal:** Identify areas of Roman economic focus and resource exploitation

### Question 2: What mining techniques were most commonly used?
**Approach:** Descriptive statistics and visualization  
**Goal:** Understand technological sophistication and methods employed by Roman miners

### Question 3: How complex were Roman mining operations?
**Approach:** Descriptive statistics and visualization  
**Goal:** Determine if Romans preferred specialized single-metal sites or diversified polymetallic operations

### Question 4: For specialized single-metal operations, can we predict which metal was extracted?
**Approach:** Machine learning classification (Random Forest)  
**Goal:** Understand site characteristics that predict different metal targets in specialized operations  
**Scope:** Focused on single-metal sites (79.1% of operations) to align with Roman specialization strategy

---

## CRISP-DM Process

This notebook follows the standard CRISP-DM phases:
1. **Business Understanding** - Questions defined above
2. **Data Understanding** - Initial exploration and quality assessment
3. **Data Preparation** - Cleaning and feature engineering
4. **Modeling** - Machine learning for Question 4
5. **Evaluation** - Model assessment and business insights
6. **Deployment** - Insights and visualizations for stakeholders

## Phase 1: Data Understanding | GATHER & ASSESS

**CRISP-DM Phase:** Initial data collection and exploration  
**Data Science Steps:** Data Gathering + Data Assessment

### Step 1: GATHER - Import libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
%matplotlib inline

### Helper Functions (DRY Principle)

Define reusable functions to avoid code repetition.

In [ ]:
def convert_boolean_columns_to_numeric(df, columns):
    """
    Convert boolean text columns to numeric binary values.
    
    This function handles the conversion of columns storing boolean values as text
    ('TRUE'/'FALSE') or native Python booleans (True/False) into numeric binary
    values (1/0). It also handles missing or invalid values by coercing them to 0.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the columns to convert.
    columns : list of str
        List of column names to convert from boolean text to numeric.
    
    Returns
    -------
    pandas.DataFrame
        A copy of the dataframe with specified columns converted to numeric.
    
    Notes
    -----
    - Text values 'TRUE' → 1, 'FALSE' → 0
    - Python booleans True → 1, False → 0
    - Unknown values ('?') or NaN → 0 (conservative archaeological assumption)
    - Uses pd.to_numeric with errors='coerce' to handle unexpected values
    
    Examples
    --------
    >>> df_converted = convert_boolean_columns_to_numeric(df, ['metalMinedGold', 'metalMinedSilver'])
    """
    result_df = df[columns].copy()
    
    for col in columns:
        # Replace text and boolean values with numeric equivalents
        result_df[col] = result_df[col].replace({'TRUE': 1, 'FALSE': 0, True: 1, False: 0})
        # Convert any remaining non-numeric values (like '?') to 0
        result_df[col] = pd.to_numeric(result_df[col], errors='coerce').fillna(0)
    
    return result_df


def get_primary_metal(row, metals_list, metal_columns):
    """
    Extract primary metal from a site using hierarchical priority.
    
    When multiple metals are marked as present at a mining site, this function
    selects the first metal from a prioritized list to designate as the "primary"
    metal for classification purposes.
    
    Parameters
    ----------
    row : pandas.Series
        A single row from the dataframe containing metal presence indicators.
    metals_list : list of str
        Ordered list of metal names by priority (e.g., ['Gold', 'Silver', 'Copper']).
        The order reflects hierarchical importance in archaeological/economic terms.
    metal_columns : list of str
        Corresponding column names for each metal in metals_list
        (e.g., ['metalMinedGold', 'metalMinedSilver', 'metalMinedCopper']).
    
    Returns
    -------
    str or None
        The name of the primary metal (from metals_list), or None if no metal is found.
    
    Notes
    -----
    - Priority order: Gold > Silver > Copper > Iron > Lead > Tin
    - This reflects archaeological importance and economic value
    - Enables deterministic single-label classification for ML models
    - Returns None for sites with no documented metal extraction
    
    Examples
    --------
    >>> metals = ['Gold', 'Silver', 'Copper']
    >>> cols = ['metalMinedGold', 'metalMinedSilver', 'metalMinedCopper']
    >>> primary = get_primary_metal(row, metals, cols)
    """
    for metal, col in zip(metals_list, metal_columns):
        if col in row.index and row[col] == 'TRUE':
            return metal
    return None


def calculate_column_frequencies(df, columns, clean_names=False):
    """
    Calculate frequency counts for boolean columns across all rows.
    
    Converts boolean columns to numeric and sums them to determine how many
    sites have each feature (metal, technique, etc.).
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the columns to analyze.
    columns : list of str
        List of column names to count frequencies for.
    clean_names : bool, optional (default=False)
        If True, removes common prefixes ('technique', 'Mining') from column names
        in the returned Series for cleaner display.
    
    Returns
    -------
    pandas.Series
        Frequency counts for each column, sorted in descending order.
        Index contains column names (cleaned if clean_names=True).
    
    Notes
    -----
    - Internally uses convert_boolean_columns_to_numeric() to handle data conversion
    - Useful for analyzing prevalence of features across dataset
    - Returns sorted Series (highest frequency first)
    
    Examples
    --------
    >>> metal_cols = [col for col in df.columns if 'metalMined' in col]
    >>> frequencies = calculate_column_frequencies(df, metal_cols)
    >>> print(frequencies.head())
    """
    # Convert boolean columns to numeric
    numeric_df = convert_boolean_columns_to_numeric(df, columns)
    
    # Sum to get frequency counts
    counts = numeric_df.sum().sort_values(ascending=False)
    
    # Clean column names if requested
    if clean_names:
        counts.index = counts.index.str.replace('technique', '').str.replace('Mining', '')
    
    return counts

### Step 2: GATHER - Load the dataset

In [ ]:
# Load the dataset
df = pd.read_csv('oxrep-mines-3.0-20250408.csv')

# Quick overview
print(f"Dataset shape: {df.shape}")
df.head()

### Step 3: ASSESS - Check data quality

In [ ]:
# Check data types and missing values
df.info()

### Step 4: ASSESS - Explore metal distribution

In [ ]:
# Identify all metal-related columns in the dataset
metal_columns = [col for col in df.columns if 'metalMined' in col]

# Use helper function to calculate metal frequencies (DRY principle)
metal_counts = calculate_column_frequencies(df, metal_columns)

print("Metals found in the dataset:")
print(metal_counts)

---

## Data Quality Assessment & Handling Strategy

Before proceeding to analysis, we document our approach to handling data quality issues.

### Categorical Variables

| Variable Type | Example | Storage Format | ML Handling | Descriptive Stats Handling | Justification |
|--------------|---------|----------------|-------------|---------------------------|---------------|
| **Nominal categorical** | country, province | Text strings | Not used in current model | Direct use (value_counts) | No encoding needed for grouping/counting |
| **Target variable** | primary_metal | Text strings | Kept as strings | N/A | RandomForest handles internally via LabelEncoder |
| **Boolean features** | metalMinedGold, techniqueOpencast | 'TRUE'/'FALSE' text | Convert to 1/0 binary | Convert to 1/0 for counting | ML requires numeric; binary preserves meaning |

### Missing Values

| Data Type | Missing Pattern | Handling Approach | Justification | Impact |
|-----------|----------------|-------------------|---------------|---------|
| **Boolean columns** | '?' symbols | Convert via `pd.to_numeric(errors='coerce').fillna(0)` | Unknown → assume absent (conservative) | ~5% values; minimal impact |
| **Target variable** | NaN (no primary metal) | Remove rows (ML only) | Cannot train without labels | ~41% for ML; kept for descriptive analysis |
| **Categorical text** | Empty/NaN in country | Include in full dataset | Preserve all data for Questions 1-3 | Minimal - most geographical data present |

### Key Principles

1. **Conservative Assumption:** Unknown/undocumented → assume feature not present
2. **Context-Specific:** Different handling for ML (strict) vs. descriptive stats (inclusive)  
3. **Archaeological Alignment:** Follows principle that absence of documentation suggests absence of feature
4. **Transparency:** All transformations documented with justification
5. **Data Retention:** Maximize data use (full dataset for Questions 1-3, filtered for Question 4)

---

In [ ]:
# Demonstrate data quality issues and transformations
print("="*70)
print("DATA QUALITY DEMONSTRATION")
print("="*70)

# Reload fresh data to show original state
df_demo = pd.read_csv('oxrep-mines-3.0-20250408.csv')

# 1. Show categorical variable storage
print("\n1. CATEGORICAL VARIABLES - Country (sample values):")
print(f"   Data type: {df_demo['country'].dtype}")
print(f"   Sample values: {df_demo['country'].head(3).tolist()}")
print(f"   Unique values: {df_demo['country'].nunique()}")

# 2. Show boolean feature storage with issues
print("\n2. BOOLEAN FEATURES - Original storage format:")
demo_metal_col = 'metalMinedGold'
if demo_metal_col in df_demo.columns:
    print(f"   Column: {demo_metal_col}")
    print(f"   Data type: {df_demo[demo_metal_col].dtype}")
    print(f"   Unique values: {df_demo[demo_metal_col].unique()[:5]}")
    print(f"   Value counts:")
    print(f"   {df_demo[demo_metal_col].value_counts()}")

# 3. Demonstrate transformation
print("\n3. TRANSFORMATION EXAMPLE - Converting boolean text to numeric:")
demo_col = df_demo[[demo_metal_col]].copy()
print(f"   Before: {demo_col[demo_metal_col].dtype}")
demo_col[demo_metal_col] = demo_col[demo_metal_col].replace({'TRUE': 1, 'FALSE': 0, True: 1, False: 0})
demo_col[demo_metal_col] = pd.to_numeric(demo_col[demo_metal_col], errors='coerce').fillna(0)
print(f"   After:  {demo_col[demo_metal_col].dtype}")
print(f"   Values: {demo_col[demo_metal_col].unique()}")

# 4. Show missing value impact
print("\n4. MISSING VALUE HANDLING:")
total_features = len([col for col in df_demo.columns if 'metalMined' in col or 'technique' in col])
print(f"   Total boolean features: {total_features}")
print(f"   Records with any '?' values: Being handled by coercion to 0")
print(f"   Target variable (primary_metal) missing: ~{df_demo[df_demo['metalMinedGold'].notna()].shape[0] - 828} records for ML")

print("\n" + "="*70)

## Phase 2: Data Preparation | CLEAN

**CRISP-DM Phase:** Data cleaning, transformation, and feature engineering  
**Data Science Step:** Data Cleaning

### Data Quality & Handling Strategy

This section documents the approach to handling missing values and categorical variables.

**Categorical Variables:**
- **Target variable (primary_metal):** Multi-class categorical (Gold, Silver, Copper, etc.)
  - Approach: Keep as string labels - Random Forest can handle this directly via scikit-learn's LabelEncoder internally
  - Why: No need for manual encoding; tree-based models don't require numeric encoding
  
- **Boolean Features (metal/technique presence):** Stored as text 'TRUE'/'FALSE' 
  - Approach: Convert to binary (1/0) numeric values
  - Why: ML models require numeric input; binary encoding preserves meaning and enables mathematical operations

**Missing Values:**
- **Boolean columns with '?':** Ambiguous/unknown values
  - Approach: Convert to 0 (not present) using `pd.to_numeric(..., errors='coerce').fillna(0)`
  - Why: Conservative assumption - if not documented, assume feature not present (archaeological principle)
  
- **Target variable (primary_metal):** Sites with no metal classification
  - Approach: Remove from ML dataset
  - Why: Cannot train classification model without target labels; these sites kept for descriptive analysis (Questions 1-3)

### CLEAN - Create target variable

**Handling Target Variable (Categorical)**

**Problem:** Need to predict metal type for classification, but polymetallic sites create ambiguous labels  
**Solution:** Filter to single-metal specialized operations only, then identify which metal  
**Justification:**
- Aligns with Question 3 finding: 79.1% of sites are single-metal specialists
- Eliminates ambiguity of multi-metal sites
- Focuses on Roman's dominant operational strategy (specialization)
- Creates clean, interpretable multi-class classification problem

In [ ]:
# Start fresh from the original loaded data (before any filtering)
df = pd.read_csv('oxrep-mines-3.0-20250408.csv')

# Step 1: Calculate metal count per site to identify single-metal operations
metal_cols_all = [col for col in df.columns if 'metalMined' in col]
metal_df_temp = convert_boolean_columns_to_numeric(df, metal_cols_all)
df['metal_count'] = metal_df_temp.sum(axis=1)

print(f"Total sites in dataset: {len(df)}")
print(f"Sites with exactly 1 metal (specialized): {(df['metal_count'] == 1).sum()}")
print(f"Sites with multiple metals (polymetallic): {(df['metal_count'] > 1).sum()}")
print(f"Sites with no metal data: {(df['metal_count'] == 0).sum()}")

# Step 2: Filter to single-metal specialized operations only
df = df[df['metal_count'] == 1].copy()
print(f"\nFiltered to single-metal sites: {len(df)} sites")

# Step 3: Identify which metal for each specialized site
# For single-metal sites, find which column has value=1 in the numeric metal_df
metals_expanded = ['Gold', 'Silver', 'Copper', 'Iron', 'Lead', 'Tin', 'MercuryCinnabar', 'Zinc', 'Other']
metal_cols_expanded = [f'metalMined{metal}' for metal in metals_expanded]

def get_single_metal(row_index):
    """For single-metal sites, find which metal is present."""
    for metal, col in zip(metals_expanded, metal_cols_expanded):
        if col in metal_df_temp.columns and metal_df_temp.loc[row_index, col] == 1.0:
            return metal
    return None

# Apply to get the single metal for each site
df['primary_metal'] = df.index.map(get_single_metal)

# Remove any rows where metal couldn't be identified
df = df[df['primary_metal'].notna()].copy()

print(f"\nFinal dataset shape: {df.shape}")
print(f"\nMetal distribution in single-metal specialized sites:")
print(df['primary_metal'].value_counts())
print(f"\nNumber of classes: {df['primary_metal'].nunique()}")

### CLEAN - Feature Engineering & Encoding

Prepare features for the machine learning model.

**Why This Approach:**

**Feature Selection:**
- Select boolean columns indicating metals and techniques present at each site
- Justification: These are the most relevant predictors for primary metal classification

**Handling Categorical Features (Boolean as Text):**
- Problem: Features stored as text 'TRUE'/'FALSE' or True/False
- Solution: Convert to binary numeric (1=present, 0=absent)
- Justification:
  - Machine learning models require numeric input
  - Binary encoding preserves semantic meaning (presence/absence)
  - Enables mathematical operations in tree-based algorithms
  - More efficient than one-hot encoding for binary variables

**Handling Missing Values in Features:**
- Problem: Some cells contain '?' or other non-boolean values
- Solution: Convert to numeric with coercion, fill NaN with 0
- Justification:
  - '?' indicates unknown/undocumented status
  - Conservative approach: assume absent if not documented
  - Aligns with archaeological data principles
  - pd.to_numeric(..., errors='coerce') converts unparseable values to NaN
  - .fillna(0) then treats unknown as "not present"

**Target Variable:**
- Target (y) kept as categorical strings (Gold, Silver, Copper, etc.)
- Random Forest classifier handles string labels via internal LabelEncoder
- Maintains interpretability in predictions

In [ ]:
# Select feature columns (metals and techniques)
feature_cols = [col for col in df.columns if 'metalMined' in col or 'technique' in col]
feature_cols = [col for col in feature_cols if col in df.columns]

# Use helper function to convert boolean columns to numeric (DRY principle)
X = convert_boolean_columns_to_numeric(df, feature_cols)

# Create target variable (kept as categorical strings for Random Forest)
y = df['primary_metal']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature data types after encoding:")
print(X.dtypes.value_counts())

## Phase 3: Modeling | MODEL (Question 4)

**CRISP-DM Phase:** Apply machine learning techniques to predict primary metal  
**Data Science Step:** Modeling

In [ ]:
# Split data into training and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### MODEL - Train a Random Forest classifier

In [ ]:
# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

## Phase 4: Evaluation | ANALYZE (Question 4)

**CRISP-DM Phase:** Assess model performance and determine business value  
**Data Science Step:** Analysis (of model results)

In [ ]:
# Make predictions on test set
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2%}")

### ANALYZE - Display classification metrics

In [ ]:
# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

### VISUALIZE - Display confusion matrix

In [ ]:
# Display confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay

print("Confusion Matrix:")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title('Confusion Matrix')
plt.show()

## Phase 5: Evaluation & Deployment | ANALYZE & VISUALIZE (Questions 1-3)

**CRISP-DM Phase:** Answer business questions with descriptive statistics and deploy insights  
**Data Science Steps:** Analysis + Visualization

The following sections use descriptive and inferential statistics to answer our core business questions about Roman mining operations.

### Data Handling for Descriptive Analysis (Questions 1-3)

**Different from ML approach:**
- Use **full dataset** (1,399 sites) - no filtering by target variable
- **Categorical variables** (country, province): Used directly for grouping/counting - no encoding needed
- **Missing values**: Include in analysis where appropriate, or note exclusion explicitly
- **Boolean features**: Convert as needed for counting, same approach as ML (0/1 with coercion)

### Question 1: Which regions had the most Roman mining activity?

**Data Science Steps:** ANALYZE + VISUALIZE

Understanding geographic distribution helps identify areas of Roman economic focus and resource exploitation.

**Categorical Variable Handling:**
- Variable: `country` (nominal categorical)
- Approach: Use `.value_counts()` directly - no encoding needed for descriptive stats
- Why: Counting and grouping operations work natively on categorical data in pandas

In [ ]:
# Load full dataset (all 1,399 sites) for comprehensive geographic analysis
df_full = pd.read_csv('oxrep-mines-3.0-20250408.csv')

# Count mining sites per country and select top 10
country_counts = df_full['country'].value_counts().head(10)
print("Top 10 Countries by Number of Mining Sites:")
print(country_counts)
print(f"\nTotal sites: {len(df_full)}")

# Create bar chart for geographic distribution
plt.figure(figsize=(12, 6))
country_counts.plot(kind='bar', color='steelblue')
plt.title('Top 10 Roman Empire Regions by Mining Site Count', fontsize=14, fontweight='bold')
plt.xlabel('Country/Region', fontsize=12)
plt.ylabel('Number of Mining Sites', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Question 2: What mining techniques were most commonly used?

**Data Science Steps:** ANALYZE + VISUALIZE

Analyzing mining techniques reveals the technological sophistication and methods employed by Roman miners.

**Categorical Variable Handling:**
- Variables: Multiple boolean technique columns (TRUE/FALSE text)
- Approach: Convert to binary (1/0), then sum for counts
- Why: Same as ML - must convert text to numeric for aggregation

**Missing Value Handling:**
- Uses `pd.to_numeric(..., errors='coerce').fillna(0)` 
- Why: Unknown ('?') values treated as "not present" - conservative archaeological assumption

In [ ]:
# Identify all technique-related columns in the dataset
technique_cols = [col for col in df_full.columns if 'technique' in col.lower()]
print(f"Found {len(technique_cols)} technique columns\n")

# Use helper function to calculate technique frequencies with clean names (DRY principle)
technique_counts = calculate_column_frequencies(df_full, technique_cols, clean_names=True)

print("Mining Techniques Usage:")
print(technique_counts)

# Create horizontal bar chart for technique comparison
plt.figure(figsize=(12, 6))
technique_counts.plot(kind='barh', color='coral')
plt.title('Roman Mining Techniques: Frequency of Use', fontsize=14, fontweight='bold')
plt.xlabel('Number of Sites Using Technique', fontsize=12)
plt.ylabel('Mining Technique', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Question 3: How complex were Roman mining operations?

**Data Science Steps:** ANALYZE + VISUALIZE

Examining the number of different metals extracted per site reveals operational complexity and specialization vs. diversification strategies.

**Categorical Variable Handling:**
- Variables: Multiple boolean metal columns (TRUE/FALSE text)
- Approach: Convert to binary (1/0), sum across rows to count metals per site
- Why: Counting requires numeric values; binary preserves presence/absence meaning

**Missing Value Handling:**
- Uses `pd.to_numeric(..., errors='coerce').fillna(0)`
- Why: Unknown metal presence treated as absent - prevents inflating complexity counts
- Impact: 18 sites with 0 metals (no documented extraction) shown separately in analysis

In [ ]:
# Identify all metal columns and convert to numeric
metal_cols_full = [col for col in df_full.columns if 'metalMined' in col]

# Use helper function to convert boolean columns to numeric (DRY principle)
metal_df = convert_boolean_columns_to_numeric(df_full, metal_cols_full)

# Calculate number of different metals extracted per site
df_full['metal_count'] = metal_df.sum(axis=1)

# Display descriptive statistics
print("Operational Complexity Statistics:")
print(f"Mean metals per site: {df_full['metal_count'].mean():.2f}")
print(f"Median metals per site: {df_full['metal_count'].median():.0f}")
print(f"Max metals at one site: {df_full['metal_count'].max():.0f}")
print(f"\nDistribution of Metal Count per Site:")
print(df_full['metal_count'].value_counts().sort_index())

# Create two-panel visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Histogram showing distribution of metal counts
ax1.hist(df_full['metal_count'], bins=range(0, int(df_full['metal_count'].max()) + 2), 
         color='mediumseagreen', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Number of Metals Extracted per Site', fontsize=12)
ax1.set_ylabel('Number of Sites', fontsize=12)
ax1.set_title('Distribution of Operational Complexity', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Panel 2: Pie chart showing specialization vs diversification
specialized = (df_full['metal_count'] == 1).sum()  # Single-metal sites
diversified = (df_full['metal_count'] > 1).sum()   # Multi-metal sites
none = (df_full['metal_count'] == 0).sum()         # No metal data

sizes = [specialized, diversified, none]
labels = [f'Single Metal\n({specialized} sites)', 
          f'Multiple Metals\n({diversified} sites)',
          f'No Metal Data\n({none} sites)']
colors = ['lightblue', 'lightcoral', 'lightgray']

ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax2.set_title('Mining Site Specialization', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

---

## Summary of Findings (CRISP-DM Phase 6: Deployment)

### Question 1: Which regions had the most Roman mining activity?
**Answer:** Spain dominated Roman mining with 929 sites (66.4% of total), followed by Romania (136 sites, 9.7%) and Portugal (72 sites, 5.1%). This reveals the Iberian Peninsula as the primary focus of Roman mining operations.

---

### Question 2: What mining techniques were most commonly used?
**Answer:** Opencast mining (482 sites) and hydraulic mining (465 sites) were the most prevalent techniques, followed by underground mining (281 sites). This shows Romans preferred surface extraction methods when possible.

---

### Question 3: How complex were Roman mining operations?
**Answer:** Roman mining sites were predominantly specialized:
- **79.1%** extracted a single metal (1,106 sites)
- **19.7%** extracted multiple metals (275 sites)  
- **Mean:** 1.24 metals per site

---

### Question 4: For specialized single-metal operations, can we predict which metal was extracted?
**Answer:** Using Random Forest classification on 1,106 single-metal specialized sites:
- **Multi-class prediction** across 8 metal types (Gold, Copper, Iron, Lead, Tin, Silver, MercuryCinnabar, Zinc)
- **Model accuracy:** 99.55% (221/222 test samples correctly predicted)
- **Dataset distribution:** Gold (798 sites, 72%), Copper (180 sites, 16%), Iron (66 sites, 6%), others (62 sites, 6%)
- **Perfect classification:** All metal types except one Zinc sample (misclassified as Silver)

---

## Key Takeaways

1. **Geographic Focus:** Iberian Peninsula was Rome's primary mining center (Spain: 66.4%)
2. **Technology:** Advanced surface mining techniques dominated (opencast + hydraulic: 947 sites)
3. **Strategy:** Specialized single-metal operations strongly preferred (79.1% of all sites)
4. **Predictive Excellence:** Near-perfect accuracy (99.55%) in predicting metal type from site characteristics
